In [25]:
import requests

supported_ontologies = [
    'cl', 'efo', 'hancestro', 'mco', 'ms', 'pride_cv', 'obi',
    'fbbt', 'po', 'clo', 'uberon', 'zfa', 'zfs', 'fbcv',
    'micro', 'panto', 'rs', 'chebi', 'mondo'
]

def search_ols_term(term, exact=True):
    url = "https://www.ebi.ac.uk/ols4/api/search"
    params = {
        "q": term,
        "type": "class",
        "rows": 100
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    results = response.json()

    if results['response']['numFound'] == 0:
        return []

    seen = set()
    filtered = []
    for doc in results['response']['docs']:
        label = doc.get('label', '').strip().lower()
        obo_id = doc.get('obo_id')
        ontology = doc.get('ontology_name')
        description = doc.get('description')

        if ontology not in supported_ontologies:
            continue

        if exact and label != term.lower():
            continue

        if obo_id and obo_id not in seen:
            seen.add(obo_id)
            filtered.append(doc)

    return filtered

# Example usage
term_info = search_ols_term("tandem mass spectrometer", exact=True)
for result in term_info:
    print(f"{result.get('label')} ({result.get('obo_id')}) - {result.get('ontology_name')}: {result.get('description')}")


tandem mass spectrometer (OBI:0001088) - obi: ['A mass spectrometer in which ions are subjected to two or more sequential stages of analysis (which may be separated spatially or temporally) according to the quotient mass/charge.']


In [33]:
selected_onto = "OBI:0001088"
term = selected_onto.replace(':', '_')
onto = 'obi'
encoded_url = f"https://www.ebi.ac.uk/ols4/api/ontologies/{onto}/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252F{term}/hierarchicalChildren?lang=en"
response = requests.get(encoded_url)
response.raise_for_status()
data = response.json()
#check if _embedded is in the data, then it can be parsed and checked for children
if '_embedded' in data:
    for element_idx in range(len(data["_embedded"]['terms'])):
        label = data["_embedded"]['terms'][element_idx]['label']
        description = data["_embedded"]['terms'][element_idx]['description'][0]
        child_id = data["_embedded"]['terms'][element_idx]['obo_id'][0]
        print(label)
else:
    print('no children found')

no children found
